In [3]:
import os
from utils.utils import slice_video, process_video

In [4]:
root_dir = os.path.dirname(os.path.abspath("__file__"))

In [9]:
vid_dir = os.path.join(root_dir, "src", "test_videos")
pictures_dir = os.path.join(root_dir, "src", "pictures")

In [10]:
video_files = [f for f in os.listdir(vid_dir) if f.endswith('.avi')]
write_path = os.path.join(pictures_dir, "temp")
print(video_files)

for video_name_ext in video_files:
    slice_video(video_name_ext, 1, vid_dir, write_path=write_path)

['test_medium.avi']
end of the video: test_medium


In [11]:
base_write_path = os.path.join(pictures_dir, "test")

In [13]:
base_write_path

'd:\\Programming\\Work\\sewerbot_ai\\src\\pictures\\test'

# test_medium

In [14]:
end_frames=[(1921, 2169)]
process_video('test_medium.avi', special_frames=end_frames, src=vid_dir, base_write_path=base_write_path)

end of the video


# eval

In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

In [6]:
from utils.utils import load_model

In [15]:
model_name = 'augmented.pth'
model_dir = os.path.join(root_dir, 'models', model_name)

In [16]:
model = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)
model.fc = nn.Sequential(
    nn.Linear(model.fc.in_features, 256),
    nn.ReLU(),
    nn.Dropout(0.5),
    nn.Linear(256, 2)
)

optimizer = optim.Adam(model.parameters(), lr=0.001)

_ = load_model(model, optimizer, model_dir)
# model.load_state_dict(torch.load(model_dir))
model.eval()

Using cache found in C:\Users\TéBé/.cache\torch\hub\pytorch_vision_main


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [17]:
test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [18]:
data_directory = os.path.join(root_dir, "src", "pictures", "test")
dataset = ImageFolder(data_directory, transform=test_transform)

In [19]:
batch_size = 144*3
test_data_loader = DataLoader(dataset, batch_size=int(batch_size), shuffle=True)

In [20]:
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
model.to(DEVICE)  # Move the model to the same device as during training

model.eval()  # Set the model to evaluation mode
test_correct = 0
test_total = 0
test_running_loss = 0.0

criterion = nn.CrossEntropyLoss()


with torch.no_grad():
    for inputs, labels in test_data_loader:
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)
        test_outputs = model(inputs)
        test_loss = criterion(test_outputs, labels)

        test_running_loss += test_loss.item()
        _, test_predicted = torch.max(test_outputs, 1)
        test_total += labels.size(0)
        test_correct += (test_predicted == labels).sum().item()

test_accuracy = 100 * test_correct / test_total
test_loss = test_running_loss / len(test_data_loader)

print(f'Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.2f}%')


Test Loss: 0.3263, Test Accuracy: 95.80%


## pretrained_barebone 95.53%
## pretrained_augmented 96.40%
## barebone 95.48
## augmented 95.80%